In [2]:
# Importing modules
import gurobipy as grb
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from pandas import ExcelWriter
from collections import Counter
from itertools import combinations
import datetime
from preprocessing import course_data as prelim

# Function for saving excel files
def save_xls(list_dfs, xls_path, sheet_names):
    writer = ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer, sheet_names[n])
    writer.save()
    return

%matplotlib inline

In [ ]:
# Formatting Data for Input
termid = 20172
open_file = 'DSO 570 - Input Data - Small.xlsx'
rmcaps = pd.read_excel(open_file, sheetname = 'RoomCaps', index_col = 0)
timeslots = pd.read_excel(open_file, sheetname = 'Timeslots', index_col = 0)
term = pd.read_excel(open_file, sheetname = 'Term_' + str(termid))[['Course','Seats','Section','Timeslots','Slotslength',
                                                                    'First Room']]

# Creating dictionary for looking up course slots
course_slots_dict = {}
for idx, row in term.iterrows():
    course_slots_dict[idx] = [int(x) for x in row.Timeslots.split(' ')]

rmcaplvl_0 = []
rmcaplvl_1 = []
rmcap_cap = []

for idx, row in rmcaps.iterrows():
    for slot in timeslots.index:
        rmcaplvl_0.append(idx)
        rmcaplvl_1.append(slot)
        rmcap_cap.append(row.Capacity)

arrays = [np.array(rmcaplvl_0), np.array(rmcaplvl_1)]
# Multi index dataframe where level 0 is class room and level 1 is timeslot
roomcap_data = pd.DataFrame({'Capacity': rmcap_cap}, index=arrays)


In [ ]:
# Let I be the index for courses and time slots in the term
# I is a tuple of (course number, timeslot)

# mod.setParam('Timeslot', 600)

I = []
for i, (course_id, course_slots) in enumerate(course_slots_dict.items()):
    for course_time in course_slots:
        I.append((course_id, course_time))

# Let J be the index for rooms
J = rmcaps.index

# Let K be the index for timeslots
K = timeslots.index

mod = grb.Model()


# Creating decision variables
x={}
for i in I:
    for j in J:
        x[i[0], j, i[1]] = mod.addVar(vtype = grb.GRB.BINARY, lb=0,
                                      name='{0},{1},{2}'.format(i[0], j, i[1]))

# Setting objective function
mod.setObjective(sum(x[i[0],j,i[1]]*(term['Seats'].loc[i[0]] /
                                     rmcaps['Capacity'].loc[j]) for i in I for j in J),
                 sense = grb.GRB.MAXIMIZE)
            
# Adding seating vs room capacity constraint
# Required Seats*X <= Room Capacity*X
room_constraints = {}
for i in I:
    for j in J:
        mod.addConstr(x[i[0], j, i[1]]*term['Seats'].loc[i[0]]
                      <= x[i[0], j, i[1]]*roomcap_data.iloc[(roomcap_data.index.get_level_values(0) == j) &
                                                            (roomcap_data.index.get_level_values(1) == i[1]), 0].values[0])


# Force one class at a time
# sum of all X_ijk across courses, i, for each classroom j and timeslot k <= 1
for k in K:
    for j in J:
        curr_courses = [course_id for course_id, course_timeslots in course_slots_dict.items() if k in course_timeslots]
        if len(curr_courses) > 0:
            mod.addConstr(sum(x[i, j, k] for i in curr_courses) <= 1)

# Constraint for ensuring that each course for every listed timeslot must be used
for i in I:
    mod.addConstr(sum(x[i[0],j,i[1]] for j in J) == 1)
    
# Constraint for ensuring that only one classroom can be chosen by a course
# x_ijk == x_ijk+1
for i, (course_id, course_slots) in enumerate(course_slots_dict.items()):
    for course_time in course_slots[1:]:
        for j in J:
            mod.addConstr(x[course_id, j, course_time] == x[course_id, j, course_slots[0]])

# Calling the Gurobi solver to solve the model we inputed.
mod.optimize()

In [ ]:
# Outputting optimization results into excel file

course_code = []
room = []
ts = []
optimal_x = []
seating = []
capacity = []
course_name = []
weekday = []
timeofday = []
original_room = []
original_cap = []


for var in mod.getVars():
    curr_course_code, curr_room, curr_ts = var.varName.split(',')
    curr_opt = var.x
    if curr_opt != 0:
        capacity.append(rmcaps['Capacity'].loc[curr_room])
        seating.append(term['Seats'].loc[int(curr_course_code)])
        course_code.append(int(curr_course_code))
        room.append(curr_room)
        ts.append(int(curr_ts))
        optimal_x.append(curr_opt)
        course_name.append(term['Course'].loc[int(curr_course_code)])
        weekday.append(timeslots['Weekday'].loc[int(curr_ts)])
        timeofday.append(timeslots['Time'].loc[int(curr_ts)])
        original_room.append(term['First Room'].loc[int(curr_course_code)])
        try:
            original_cap.append(rmcaps['Capacity'].loc[term['First Room'].loc[int(curr_course_code)]])
        except:
            original_cap.append('N/A')
        
        

objective_optimal = pd.DataFrame({'Optimal': mod.ObjVal}, index = [0])
solution_df = pd.DataFrame({'Course': course_code,
                            'Course Name': course_name,
                            'Room': room,
                            'Original Room': original_room,
                            'Original Capacity': original_cap,
                            'Timeslot': ts,
                            'Day': weekday,
                            'Time': timeofday,
                            'Optimal X': optimal_x},
                           index = range(len(optimal_x)))

seat_df = pd.DataFrame({'Course': course_code,
                        'Course Name': course_name,
                        'Room': room,
                        'Original Room': original_room,
                        'Original Capacity': original_cap,
                        'Timeslot': ts,
                        'Day': weekday,
                        'Time': timeofday,
                        'Req. Seat': seating},
                       index = range(len(optimal_x)))

cap_df = pd.DataFrame({'Course': course_code,
                       'Course Name': course_name,
                       'Room': room,
                       'Original Room': original_room,
                       'Original Capacity': original_cap,
                       'Timeslot': ts,
                       'Day': weekday,
                       'Time': timeofday,
                       'Capacity': capacity},
                      index = range(len(optimal_x)))

df_lst = [objective_optimal, solution_df, seat_df, cap_df]
shtnames = ['Optimal Objective','Optimal Solution','Req. Seating','Capacities']
path = 'DSO 570 Final - Output Small.xlsx'

save_xls(df_lst,path,shtnames)